In [1]:
'''
3. Relate entities in MP's questions to constituencies
'''

"\n3. Relate entities in MP's questions to constituencies\n"

In [2]:
import pandas as pd
import geopandas as gpd

MP_geometry = pd.read_csv("../data/MP_geometry.csv")

question_locations = pd.read_csv("../data/question_locations_full.csv")

In [3]:
# converts MP_geometry geometry column to type geometry
MP_geometry['geometry'] = gpd.GeoSeries.from_wkt(MP_geometry['geometry'])
MP_geometry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669 entries, 0 to 668
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   const        669 non-null    object  
 1   person       669 non-null    object  
 2   personLabel  669 non-null    object  
 3   constLabel   669 non-null    object  
 4   geometry     669 non-null    geometry
dtypes: geometry(1), object(4)
memory usage: 26.3+ KB


In [4]:
'''
Checks for each entity which constituency it belongs and then adds it to the new constituency column
'''
import shapely
entity_const = question_locations.copy()
# help from https://stackoverflow.com/questions/7861196/check-if-a-geopoint-with-latitude-and-longitude-is-within-a-shapefile

# adds a new column to question_locations_df
entity_const["const"] = None


for index, row in entity_const.iterrows():
    point = shapely.geometry.Point(row["easting"], row["northing"])
    for index2, row2 in MP_geometry.iterrows():
        if row2["geometry"].contains(point):
            print(index)
            entity_const.at[index, "const"] = row2["constLabel"]
            break

0
1
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
223
224
225
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283


In [5]:
# show entities that do not have a constituency
# these are mainly rivers or where wikipedia has put the coordinates in the sea
entity_const[entity_const["const"].isnull()]

,question,entity,easting,northing,wikipedia,const
2,https://id.parliament.uk/oHeIeP9G,River_Tees,455114.463816,528466.390692,https://en.wikipedia.org/wiki/River_Tees,None
110,https://id.parliament.uk/kCXThQ8A,National_Cyber_Security_Centre_(Ireland),116782.669827,390144.965280,https://en.wikipedia.org/wiki/National_Cyber_S...,None
128,https://id.parliament.uk/XNaAfVIs,National_Cyber_Security_Centre_(Ireland),116782.669827,390144.965280,https://en.wikipedia.org/wiki/National_Cyber_S...,None
220,https://id.parliament.uk/eIPt42gK,English_Channel,400096.103588,33330.401104,https://en.wikipedia.org/wiki/English_Channel,None
221,https://id.parliament.uk/8kWXGOpu,English_Channel,400096.103588,33330.401104,https://en.wikipedia.org/wiki/English_Channel,None
...,...,...,...,...,...,...
2383,https://id.parliament.uk/zCGJsRDq,Humber,538740.556828,407153.441365,https://en.wikipedia.org/wiki/Humber,None
2446,https://id.parliament.uk/w0idvLLy,Humber,538740.556828,407153.441365,https://en.wikipedia.org/wiki/Humber,None
2452,https://id.parliament.uk/gChiL6fx,Humber,538740.556828,407153.441365,https://en.wikipedia.org/wiki/Humber,None
2499,https://id.parliament.uk/G7IGTcIW,English_Channel,400096.103588,33330.401104,https://en.wikipedia.org/wiki/English_Channel,None


In [6]:
entity_const[entity_const["const"].isnull()]["entity"].unique()

array(['River_Tees', 'National_Cyber_Security_Centre_(Ireland)',
       'English_Channel', 'Humber', 'Hastings',
       'Dublin_Institute_of_Technology', 'Ireland', 'River_Tyne',
       'River_Wear', 'Isles_of_Scilly', 'Portland_Harbour',
       'Neart_Na_Gaoithe', 'HMS_Dasher_(D37)', 'River_Foyle',
       'Lough_Foyle', 'Strangford_Lough',
       'Strangford_(UK_Parliament_constituency)', 'Ulster'], dtype=object)

In [7]:
import numpy as np

# pick closest entites for these
for index, row in entity_const[entity_const["const"].isnull()].iterrows():
    shortest_dist = np.inf
    
    # help from https://stackoverflow.com/questions/36972537/distance-from-point-to-polygon-when-inside

    point = shapely.geometry.Point(row["easting"], row["northing"])
    for index2, row2 in MP_geometry.iterrows():
        dist = row2["geometry"].boundary.distance(point)

        if dist < shortest_dist:
            shortest_dist = dist
            closest_const = row2["constLabel"]
    entity_const.at[index, "const"] = closest_const

In [8]:
entity_const["entity"].value_counts() 

entity
North_East_England                                 45
York                                               34
Birmingham                                         32
Broxbourne_(UK_Parliament_constituency)            31
Portland_Harbour                                   31
                                                   ..
Exeter                                              1
North_West_Leicestershire                           1
Shipley_(UK_Parliament_constituency)                1
Elmet_and_Rothwell_(UK_Parliament_constituency)     1
Downing_Street_mortar_attack                        1
Name: count, Length: 802, dtype: int64

In [9]:
for index, row in entity_const.iterrows():
    print(row["entity"])

West_Midlands_(region)
Slough
River_Tees
Hounslow
Buckingham_(UK_Parliament_constituency)
Kingston_upon_Hull
Kingston_upon_Hull
Charing_Cross
Liverpool_Walton_(UK_Parliament_constituency)
Feltham
Heston
Dulwich
West_Norwood
York
Dulwich
West_Norwood
Dulwich
West_Norwood
York
York
Dulwich
West_Norwood
Dulwich
West_Norwood
Dulwich
West_Norwood
Dulwich
West_Norwood
Dulwich
West_Norwood
Dulwich
West_Norwood
Dulwich
West_Norwood
Dulwich
West_Norwood
Hornsey
Wood_Green
Dulwich
West_Norwood
Dulwich
West_Norwood
Kingston_upon_Hull
Kingston_upon_Hull
Dulwich
West_Norwood
Dulwich
West_Norwood
Royal_College_of_Nursing
Manchester
South_Yorkshire
11_Downing_Street
Glasgow_Airport
Heathrow_Airport
Hillsborough_Stadium
Hillsborough_Stadium
RAF_Northolt
Broxbourne_(UK_Parliament_constituency)
Ashridge
Newcastle_upon_Tyne
Heathrow_Airport
Heathrow_Airport
University_of_Kent
Hillsborough_Stadium
Scunthorpe
Port_Talbot
Chipping_Barnet
Newcastle_upon_Tyne
North_East_England
Bristol
North_Somerset
South_Gl

In [19]:
entity_const.to_csv("../data/entity_const.csv", index=False)